In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-42.4"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2022, 6)
val monthlyReportForecastUntil = YearMonthTO(2022, 12)

val yearlyReportFrom = 2019
val yearlyReportTo = 2021
val yearlyReportForecastUntil = 2023

val fundName = "Expenses"
val reportViewName = "Expenses report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = client.ensureUserExists(username)
user

UserTO(id=6a71c209-44a0-472e-a356-ab5417e9be96, username=Johann-42.4)

In [3]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE)))
reportView

ReportViewTO(id=2ba9ed21-d981-4096-a7ec-96ca28c695ec, name=Expenses report, fundId=1a289583-85eb-40f7-8fb2-155e613e571a, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=false, filter=null), groupedNet=GenericReportConfigurationTO(enabled=false), grouped

In [4]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [5]:
val yearlyData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [6]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupData(
    reportData: ReportDataTO,
    plotTitle: String,
    groupItemFilter: (ReportDataGroupedBudgetItemTO) -> Boolean,
): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.spent!! }.reduce { acc, value -> acc + value }
        },
        "allocated" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value }
        },
        "left" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value }
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! + it.spent!! }
                .reduce { acc, value -> acc + value }
        },
    )
    return dataFrame
        .plot {
            line {
                val values =
                    listOf("spent", "allocated", "left").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotGroup(reportData: ReportDataTO, title: String, group: String): Plot =
    plotGroupData(reportData, title) { it.group == group }

fun plotGroupsTotal(reportData: ReportDataTO, title: String): Plot =
    plotGroupData(reportData, title) { true }



In [7]:
plotGroupsTotal(yearlyData, "Yearly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3LNkqO"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Yearly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"left":[20717.356169332597,5960.089925586293,96283.6233536247,125622.96874340222,154962.31413317975],
"spent":[-104100.44029852,-186242.29,-125457.94,-138600.22343284,-138600.22343284],
"allocated":[116495.1064678526,171513.41,215810.19,167939.56882261753,167939.56882261753]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-186242.29,215810.19]
},
"sampling":"none",
"x":1.6250976E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"left"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("3LNkqO");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 
 
 -200,000 
 
 
 
 
 
 
 -180,000 
 
 
 
 
 
 
 -160,000 
 
 
 
 
 
 
 -140,000 
 
 
 
 
 
 
 -120,000 
 
 
 
 
 
 
 -100,000 
 
 
 
 
 
 
 -80,000 
 
 
 
 
 
 
 -60,000 
 
 
 
 
 
 
 -40,000 
 
 
 
 
 
 
 -20,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 160,000 
 
 
 
 
 
 
 180,000 
 
 
 
 
 
 
 200,000 

In [8]:
plotGroupsTotal(monthlyReportData, "Monthly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="LeQUgK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[10804.199999999999,13507.939999999999,19624.12,19848.9760226764,18966.350981426,20191.3774648376,16960.832755946794,1772.2206880831932,7450.673059188692,12374.633487244693,14360.089861618191,20717.356169332594,26951.640795327094,32497.652837968093,30368.468532840492,32398.318180909897,25967.904581452596,15066.172806596,-2313.4034048121143,364.5528839551947,2360.311502122389,2095.594302814391,5135.469687338304,5960.0899255863,4357.868989044599,11541.734940831104,17559.65418195279,23695.473573135492,21628.278112079883,2780.432623606773,25883.767072612067,78890.67636629677,68767.67110734887,113412.06889443737,102806.82768464206,96283.62335362467,82914.78271640747,132984.87703741147,121314.00576990526,105334.01655773156,134275.24006994255,117511.26402668825,127072.78736002157,135506.05763779936,140224.34710539193,146178.96402861725,148909.41819544468,152751.21104284105],
"spent":[-4468.51,-4301.32,-9171.45,-6968.18282172,-9043.57,-7024.09,-11421.14,-25034.7199255,-4539.6390361,-6776.5,-9787.68,-5529.22,-4702.18,-5442.03,-35747.94,-17363.44,-18069.43,-20971.82,-27340.72,-15561.74,-8102.56,-10392.12,-8651.84,-13896.47,-15195.09,-6317.14,-7060.76,-7597.91,-16297.24,-18845.79,-16242.68,-3986.96,-10017.29,-6768.53,-10605.61,-6522.94,-13377.18,-14533.9,-11741.0,-22814.56,-20699.82,-16644.35,-12829.568333333335,-12545.142361111111,-13258.324224537038,-13528.410409915123,-14091.73377740805,-14382.244092192053],
"allocated":[6950.02,7005.06,15287.63,7193.0388443964,8165.27,8255.99,8188.18,9846.94,10218.9435174615,11703.85,11782.42,11887.54,10934.13,10999.45,33626.2,19395.19,11641.91,10066.59,9960.99,18240.71,10110.61,10124.3,11694.38,14718.95,13596.0,13500.0,13095.0,13736.0,14227.2,0.0,39343.24,57000.0,-100.35,51413.1,0.0,0.0,6.98,64604.87,68.63,6836.13,49639.42,-118.92,22391.091666666667,20978.412638888887,17976.61369212963,19483.02733314043,16822.18794423547,18224.036939588423]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792

In [9]:
plotGroup(monthlyReportData, "Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="pPOpoG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[2276.76,1868.95,3107.3199999999997,2827.383006045999,3004.77692650024,2932.557977757861,1393.0887701306563,1557.9304215891334,2728.473714216889,3399.942436029243,3350.0617509717595,3804.03593536355,4051.8546098095626,4304.489560808851,9376.383522405738,12500.526781309147,12992.340318494365,11953.950561045944,10904.932581498519,10568.131018595443,9840.672527299177,8192.258805201476,8637.032084957847,8399.961620646473,8465.5589780328,9196.923131358446,9593.208087869,9977.107606556281,10569.585315099717,7817.370812537469,11421.2413600101,17903.206178946697,15022.248603669468,21568.83098388203,18071.18112080547,14923.684657221302,13089.925354542387,20610.847109318594,18701.159392469697,17330.869032132057,21346.726152654184,18787.203907745934,19701.016407745934,20391.3074494126,20598.73357788482,21063.423550396397,21021.286020617266,21267.113696689878],
"spent":[-2305.04,-2509.31,-3347.92,-2293.91,-2108.18,-2382.69,-3750.61,-2493.78,-1587.87,-2253.23,-2992.86,-2517.62,-2158.1,-2165.51,-2324.86,-948.25,-1951.84,-3154.12,-3041.34,-3989.72,-2393.26,-3686.25,-1883.73,-3184.42,-2513.19,-1835.59,-2078.71,-2087.36,-1969.62,-2751.21,-3092.31,-3205.33,-2862.68,-1679.48,-3497.72,-3147.45,-1835.02,-2169.46,-1920.16,-2327.14,-2933.91,-2542.8,-2601.121666666667,-2560.189305555556,-2506.427581018519,-2476.739879436728,-2543.178202723122,-2463.633052950049],
"allocated":[2085.01,2101.5,4586.29,2013.973006046,2286.19,2311.56,2210.79,2658.69,2759.1623689134,2925.94,2945.53,2971.84,2405.49,2419.86,7397.74,4072.98,2444.77,2113.98,1992.2,3648.14,2022.12,2024.9,2338.92,2943.75,2583.2,2565.0,2488.0,2472.5,2560.9,0.0,6688.38,9690.0,-17.06,8226.1,0.0,0.0,1.05,9690.52,10.24,957.09,6949.5,-16.61,3514.934166666667,3250.480347222222,2713.853709490741,2941.429851948302,2501.040672943994,2709.46072902266]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,

In [10]:
plotGroup(monthlyReportData, "Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="foNWzT"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[4081.7999999999997,5574.14,8396.53,10298.344864054,12509.020880974116,14679.167538602976,17126.853459560232,1691.4331717911136,4843.990771163403,8832.467070224644,12903.16082810419,17019.49404008577,20995.0319562945,22649.34528289576,6396.346950123945,-1788.2233107671273,-9881.613598711312,-22826.53478463892,-42132.07778745176,-45201.4496607339,-42994.83310209484,-41248.10292915226,-39614.28464120015,-36848.1141354272,-41401.19719476359,-37951.904069554,-33847.95762525712,-30504.350803809775,-31463.502718967007,-43479.28251943162,-28522.268232544393,-10749.572583357414,-12639.491518000325,6657.920134716703,3058.671784755833,2095.833920474413,583.7835101235266,25957.627344673394,25135.359885753474,18278.576828585625,34148.47855611914,29577.428997304607,35668.46816397127,41017.060594526825,45330.874060962014,50161.72614960013,53787.03424562476,58014.390516318104],
"spent":[-500.0,-609.16,-1763.9,-400.0,-400.0,-467.23,-420.0,-18881.0,-423.24,-456.78,-400.0,-400.0,-400.0,-2736.62,-29698.21,-16136.26,-12866.69,-17070.89,-23488.88,-10712.07,-3563.87,-2752.85,-3681.19,-3840.81,-10690.64,-2616.15,-1842.52,-2704.57,-7215.39,-12018.77,-1917.23,-6744.06,-1847.5,-2296.12,-3599.27,-962.83,-1514.94,-1114.29,-850.75,-9590.88,-3986.17,-4523.22,-3245.605,-3356.302916666667,-3073.989826388889,-3176.197311921296,-3249.537087914738,-3220.392678574299],
"allocated":[2085.01,2101.5,4586.29,2301.814864054,2612.92,2641.91,2865.93,3446.42,3576.6108657802,4447.46,4477.32,4517.28,4373.62,4399.82,13450.4,7952.09,4773.14,4127.3,4183.57,7661.16,4246.46,4556.1,5262.56,6623.49,6118.2,6075.0,5892.8,6043.8,6259.9,0.0,16917.63,24510.0,-43.15,21593.5,0.0,0.0,2.86,26488.14,28.19,2734.42,19855.79,-47.65,9336.644166666667,8704.895347222222,7387.803292824074,8007.049400559414,6874.845183939365,7447.748949267645]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12

In [11]:
plotGroup(monthlyReportData, "Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="kLIrGN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[559.4200000000001,701.85,252.11000000000004,473.89015295239994,490.7168934967315,720.3290548228103,1087.5252205321503,400.89186372942095,886.6776057824845,1818.2442251063956,492.7801524895514,-338.9060059642032,255.33580292855564,1304.0077256514,4772.089013881449,7099.210393338212,7276.676339705343,8282.836605635237,8989.391153037875,11364.560218212593,11851.339064857868,12879.298509782193,11594.182067985046,9714.144013927082,11443.551008965029,13290.15258919847,14822.389454806102,17292.97279700403,19615.84874346839,19380.174510413253,25688.752977900807,35784.06846169913,33978.79483072587,42872.01973860202,42319.53914864092,42029.2703390697,41590.2637216606,52817.43447887197,52566.2497285504,51918.88411046342,59920.85433573811,58975.96926115717,62274.6750944905,65324.16974726828,67786.00562111087,70603.23115110701,72914.11630860284,75463.62689588997],
"spent":[-204.24,-207.85,-1214.12,-281.75,-554.64,-348.18,-288.04,-1474.32,-331.47,-238.42,-2502.33,-2020.41,-608.51,-161.2,-230.49,0.0,-1218.9,-202.81,-588.5,0.0,-444.43,-405.02,-2905.92,-3945.53,-304.89,-181.04,-413.17,0.0,-240.1,-233.81,-792.5,-158.6,-1784.78,-360.99,-552.62,-290.16,-440.85,-401.27,-264.13,-1945.52,-1430.16,-921.8,-778.615,-777.4579166666667,-829.0294097222222,-749.3835271990741,-781.7496544656636,-800.8437923378023],
"allocated":[347.5,350.28,764.38,503.5301529524,571.57,577.97,655.15,787.74,817.4484968668,1170.39,1178.23,1188.76,1202.79,1209.98,3698.87,2327.42,1397.0,1207.99,1294.97,2371.23,1314.38,1417.4,1637.18,2060.66,2039.4,2025.0,1964.2,2472.5,2560.9,0.0,7081.74,10260.0,-18.06,9254.3,0.0,0.0,1.25,11628.88,12.35,1298.83,9431.33,-22.77,4077.320833333333,3826.952569444444,3290.865283564815,3566.609057195216,3092.634811961484,3350.354379624941]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.

In [12]:
plotGroup(monthlyReportData, "Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iootzS"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[378.09000000000003,728.37,882.98,-167.0038375948,-614.0674478845324,-2613.504906247843,-2503.897533515344,-1814.4646858202027,-1181.2285985961446,-703.8912163643053,121.40685281376318,953.5579444109983,1459.7154002632842,2119.047539140507,2876.402124500259,4039.932129373915,4738.111309860019,5273.749597021942,5771.8534290425205,6256.472599978445,6551.057365316032,6964.569205451607,7346.433531973067,7045.16520285982,7512.26425783192,7244.008356693043,6463.344860866109,5530.7290968084435,5702.863036875234,4763.551053404556,4922.124353077997,7472.288522046854,5284.670340792734,8787.25710654398,8477.065680701728,8224.383884956773,7434.27999942126,7869.8014845577745,5598.880176802091,5018.108216090916,5964.155560145869,4278.2941995480505,4237.59169954805,4180.678991214718,3906.4127238536066,3791.5501008790698,3375.2322593233216,2950.0729309712606],
"spent":[-385.57,0.0,-609.77,-1481.52282172,-936.99,-2494.99,-463.18,0.0,-82.99,-342.57,0.0,0.0,-149.99,0.0,-1259.78,0.0,0.0,-69.0,0.0,-429.95,0.0,0.0,-77.0,-893.12,-209.0,-944.99,-1425.22,-1755.98,-682.13,-938.77,-2600.22,-1438.67,-2180.08,-96.3,-310.22,-252.66,-790.78,-4732.84,-2276.51,-1196.03,-3521.8,-1675.27,-1755.948333333333,-1685.592361111111,-1706.169224537037,-1666.676659915123,-1797.541381574717,-1921.484830039277],
"allocated":[347.5,350.28,764.38,431.5389841252,489.89,495.33,573.1,689.31,715.3536654211,819.37,824.77,832.16,656.04,659.95,2017.62,1163.71,698.54,604.0,498.05,912.04,505.53,404.9,467.72,588.79,679.8,675.0,654.8,824.2,853.6,0.0,2754.04,3990.0,-7.03,3598.9,0.0,0.0,0.56,5168.44,5.5,615.33,4467.77,-10.56,1715.245833333333,1628.679652777778,1431.902957175926,1551.814036940586,1381.223540018969,1496.325501687216]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E

In [13]:
plotGroup(monthlyReportData, "Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="a2cI0e"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[1259.7299999999993,1740.479999999999,2795.73,1621.7991370775999,447.14575118203993,1520.273702945076,1490.6455579752574,494.4524975451657,-239.94119631411616,290.94796133389434,1233.3596880450248,2084.28918850072,2161.55406575435,2820.589133828737,3550.1830155919374,4519.6808576276535,5101.417190190235,5605.434419528502,5997.851637235841,6380.12983842379,5950.435336691998,6464.457615600432,7040.967850074258,7779.892216278933,8591.60530666271,9403.588555982244,9472.991274230433,10073.266337948056,11070.3755899306,11069.32349372118,11884.90899552394,14816.713992343326,12357.67632564572,15161.205378670415,13321.254679455498,12826.2204286097,5696.011513264466,6305.261356589388,522.7324165496443,134.78569802821474,4091.5977754789164,1321.276261125211,508.324594458544,-439.418044430345,-1710.690903226641,-2883.067333589295,-4386.771799815503,-5862.451638227229],
"spent":[-1031.16,-570.0,-1237.89,-2109.0,-2235.76,0.0,-848.65,-1980.8199255,-1756.0690361,-405.5,0.0,-100.0,-579.01,0.0,-1287.38,0.0,0.0,0.0,-6.0,-350.0,-619.0,0.0,0.0,0.0,0.0,0.0,-703.0,-360.0,0.0,0.0,-2342.87,-1625.38,-2450.0,-1823.64,-1840.0,-495.0,-7131.09,-5851.14,-5789.46,-1139.91,-1503.38,-2757.03,-2895.741666666667,-2941.814305555556,-3051.517164351852,-3101.643594714506,-3208.143894274048,-3322.155885463552],
"allocated":[1042.5,1050.75,2293.14,935.0691370776,1061.46,1073.29,818.84,984.7,1021.9130824088,936.28,942.63,951.02,656.04,659.95,2017.62,969.72,582.14,503.33,398.36,729.65,404.42,506.2,584.73,735.99,815.8,810.0,785.7,961.5,995.9,0.0,3147.41,4560.0,-8.03,4627.2,0.0,0.0,0.7,6460.45,6.86,751.97,5460.19,-13.27,2082.79,1994.071666666667,1780.244305555556,1929.267164351852,1704.43942804784,1846.476047051826]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,

In [14]:
plotGroup(monthlyReportData, "Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Io2cDG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[1832.68,2281.2799999999997,3967.97,4429.185574024001,3478.100465784087,3137.589981821141,-1523.0146998925952,-644.6199768486031,128.70787994764711,1161.9293262283554,-1671.052369638862,-1092.3397537633564,-765.6328488515093,4.451224438079174,3225.1302047868116,4966.198387483957,4124.93484010972,4657.150524174535,5437.298062234171,7183.750634210058,6870.600996259288,7201.532899678172,8023.925919250063,8361.15993921261,8070.197679781706,8370.190569600973,8845.91457936621,8882.170729297588,3262.3369954126047,358.7969524191359,-3219.095022775915,8246.747436804651,9349.926585541685,11922.703170788012,11241.241940701884,9866.373037865511,8202.432334454501,11813.7757069862,11268.028847610314,11405.914023048288,10031.269822294336,6658.6670760815105,7183.597076081511,8050.458742748177,8034.135548303733,7924.473754322253,7591.27347750898,7287.098177627937],
"spent":[0.0,-392.0,-147.85,-402.0,-1930.0,-1330.0,-5561.66,-204.8,-351.0,-20.0,-3892.49,-491.19,-766.57,-330.2,-141.92,-198.28,-2005.0,-475.0,-216.0,-80.0,-1082.0,-488.0,-104.0,-843.59,-1102.37,-511.87,-298.14,-512.0,-6190.0,-2903.23,-5152.0,9185.08,1107.75,-512.0,-681.5,-1374.84,-1664.5,-264.9,-550.0,-272.1,-4353.16,-3365.44,-658.1341666666667,-283.6453472222222,-1072.705792824074,-1254.41044222608,-1316.277979078254,-1369.176144001441],
"allocated":[834.0,840.6,1834.54,863.215574024,979.88,990.75,900.68,1083.12,1124.0179138545,1053.28,1060.38,1069.78,1093.4,1099.96,3362.6,1939.55,1164.18,1006.66,996.1,1824.07,1011.06,809.9,935.55,1177.48,815.8,810.0,785.7,549.4,569.1,0.0,1573.75,2280.0,-4.01,3084.8,0.0,0.0,0.42,3876.33,4.12,410.13,2978.34,-7.11,1183.064166666667,1150.507013888889,1056.38259837963,1144.748648244599,983.0777022649821,1065.000844120397]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E1

In [15]:
plotGroup(monthlyReportData, "Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="nETU8V"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[415.72,612.8700000000001,221.48000000000013,365.3771261172001,-349.3424886266841,-185.0358848644232,-110.3680188435623,86.59739609716544,283.9928829885293,-2425.006315313534,-2069.6270411672353,-1712.775179300885,-1206.2181908716493,-704.2776287952395,171.93370155035302,1060.992942544138,1616.0381818042256,2119.5858838287595,2717.347519590719,3812.9582352687657,4291.039313792866,1641.5801962527712,2107.212874298173,1507.881068088582,1675.8889525340248,1988.775807551928,2209.7635500720558,2443.577809330869,2870.771150260344,2870.4983205427993,3708.102641419531,5417.224357813526,5413.845938973717,6442.132381234208,6317.873329580735,6317.857085427281,6318.08628294073,7610.129556414148,7521.59532216964,7186.878649383036,7682.24942067637,6822.469026033407,7151.659026033408,7438.719859366741,7607.202428811185,7789.976045709333,7902.289130682328,8034.318306069737],
"spent":[-42.5,-13.0,-850.0,0.0,-878.0,-1.0,-89.0,0.0,-7.0,-3060.0,0.0,0.0,-40.0,-48.5,-805.3,-80.65,-27.0,0.0,0.0,0.0,0.0,-3060.0,0.0,-1189.0,-375.0,-227.5,-300.0,-178.0,0.0,0.0,-345.55,0.0,0.0,0.0,-124.28,0.0,0.0,0.0,-89.99,-402.98,-1.24,-858.79,-151.9025,-135.7652083333333,-147.0789756944444,-159.3355570023148,-172.613520085841,-176.6413134263278],
"allocated":[208.5,210.15,458.61,143.8971261172,163.36,165.18,163.69,196.96,204.4371242167,351.13,353.56,356.7,546.75,549.93,1681.35,969.72,582.14,503.33,597.74,1094.42,606.64,404.9,467.72,588.79,543.8,540.0,523.8,412.1,426.9,0.0,1180.29,1710.0,-3.01,1028.3,0.0,0.0,0.14,1292.11,1.37,68.36,496.5,-0.95,481.0925,422.8260416666667,315.5615451388889,342.109173900463,284.9266050588349,308.6704888137378]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6

In [16]:
plotGroup(monthlyReportData, "Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="R0dokO"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5940.0,-8910.091553164386,-8910.044702307645,-9652.544702307645,-10456.919702307645,-11328.325952307645,-12272.349389807645,-13295.041447099311,-14402.957842498618],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5940.0,-2970.0,0.0,-742.5,-804.375,-871.40625,-944.0234375,-1022.692057291667,-1107.916395399306],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12,1.6645824E12,1.6672608E12,1.6698528E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056